Biostat/Biomath M257 Homework 3

Due May 3 @ 11:59PM

Tomoki Okuno and 805851067

System information (for reproducibility):

In [29]:
versioninfo()

Julia Version 1.10.2
Commit bd47eca2c8a (2024-03-01 10:14 UTC)
Build Info:
  Official https://julialang.org/ release
Platform Info:
  OS: macOS (arm64-apple-darwin22.4.0)
  CPU: 8 × Apple M1
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-15.0.7 (ORCJIT, apple-m1)
Threads: 1 default, 0 interactive, 1 GC (on 4 virtual cores)
Environment:
  JULIA_NUM_THREADS = 


Load packages:

In [30]:
using Pkg

Pkg.activate(pwd())
Pkg.instantiate()
Pkg.status()

  Activating project at `~/Documents/07_UCLA/Class/257/02_Homework/hw3`


Status `~/Documents/07_UCLA/Class/257/02_Homework/hw3/Project.toml`
  [6e4b80f9] BenchmarkTools v1.5.0
  [31c24e10] Distributions v0.25.108
  [37e2e46d] LinearAlgebra
  [9a3f8284] Random


In [31]:
using LinearAlgebra, Random
using BenchmarkTools, Distributions

Consider a linear mixed effects model
$$
    \mathbf{Y}_i = \mathbf{X}_i \boldsymbol{\beta} + \mathbf{Z}_i \boldsymbol{\gamma} + \boldsymbol{\epsilon}_i, \quad i=1,\ldots,n,
$$
where   
- $\mathbf{Y}_i \in \mathbb{R}^{n_i}$ is the response vector of $i$-th individual,  
- $\mathbf{X}_i \in \mathbb{R}^{n_i \times p}$ is the fixed effect predictor matrix of $i$-th individual,  
- $\mathbf{Z}_i \in \mathbb{R}^{n_i \times q}$ is the random effect predictor matrix of $i$-th individual,  
- $\boldsymbol{\epsilon}_i \in \mathbb{R}^{n_i}$ are multivariate normal $N(\mathbf{0}_{n_i},\sigma^2 \mathbf{I}_{n_i})$,  
- $\boldsymbol{\beta} \in \mathbb{R}^p$ are fixed effects, and  
- $\boldsymbol{\gamma} \in \mathbb{R}^q$ are random effects assumed to be $N(\mathbf{0}_q, \boldsymbol{\Sigma}_{q \times q}$) independent of $\boldsymbol{\epsilon}_i$.

## Q1 Formula (10 pts)

Write down the log-likelihood of the $i$-th datum $(\mathbf{y}_i, \mathbf{X}_i, \mathbf{Z}_i)$ given parameters $(\boldsymbol{\beta}, \boldsymbol{\Sigma}, \sigma^2)$.

**Solution**

Since $\boldsymbol{\gamma}$ is independent of $\boldsymbol{\epsilon}_i$, we have $\mathbf{y}_i \sim N(\mathbf{X}_i \boldsymbol{\beta}, \mathbf{\Omega}_i)$, where $\mathbf{\Omega}_i = \mathbf{Z}_i \boldsymbol{\Sigma} \mathbf{Z}_i' + \sigma^2 \mathbf{I}_{n_i}$. Hence, the likelihood $L(\boldsymbol{\beta}, \boldsymbol{\Sigma}, \sigma^2)$ is written as
$$
    L(\boldsymbol{\beta}, \boldsymbol{\Sigma}, \sigma^2) = 
    (2\pi)^{-n_i/2}\vert\mathbf{\Omega}_i\vert^{-1/2}
    \exp\left[-\frac {(\mathbf{y}_i - \mathbf{X}_i \boldsymbol{\beta})'\mathbf{\Omega}_i^{-1}(\mathbf{y}_i - \mathbf{X}_i \boldsymbol{\beta})} 2\right],
$$
and thus the log-likelihood $\ell(\boldsymbol{\beta}, \boldsymbol{\Sigma}, \sigma^2)$ is given by
$$
\begin{align*}
    \ell(\boldsymbol{\beta}, \boldsymbol{\Sigma}, \sigma^2) 
    &=
    - \frac{n_i} 2 \ln(2\pi)
    - \frac 1 2 \ln\vert\mathbf{\Omega}_i\vert
    - \frac 1 2 (\mathbf{y}_i - \mathbf{X}_i \boldsymbol{\beta})'\mathbf{\Omega}_i^{-1}(\mathbf{y}_i - \mathbf{X}_i \boldsymbol{\beta}).
\end{align*}
$$

## Q2 Start-up code

Use the following template to define a type `LmmObs` that holds an LMM datum $(\mathbf{y}_i, \mathbf{X}_i, \mathbf{Z}_i)$. 

In [32]:
# define a type that holds LMM datum
struct LmmObs{T <: AbstractFloat}
    # data
    y :: Vector{T}
    X :: Matrix{T}
    Z :: Matrix{T}
    # working arrays
    # whatever intermediate vectors/arrays you may want to pre-allocate
    storage_p  :: Vector{T}
    storage_q  :: Vector{T}
    xtx        :: Matrix{T}
    ztx        :: Matrix{T}
    ztz        :: Matrix{T}
    storage_qq :: Matrix{T}
end

# constructor
function LmmObs(
    y :: Vector{T}, 
    X :: Matrix{T}, 
    Z :: Matrix{T}
    ) where T <: AbstractFloat
    storage_p  = Vector{T}(undef, size(X, 2))
    storage_q  = Vector{T}(undef, size(Z, 2))
    xtx        = transpose(X) * X
    ztx        = transpose(Z) * X
    ztz        = transpose(Z) * Z
    storage_qq = similar(ztz)
    LmmObs(y, X, Z, storage_p, storage_q, xtx, ztx, ztz, storage_qq)
end

LmmObs

Write a function, with interface   
```julia
logl!(obs, β, L, σ²)
```
that evaluates the log-likelihood of the $i$-th datum. Here `L` is the lower triangular Cholesky factor from the Cholesky decomposition `Σ=LL'`. Make your code efficient in the $n_i \gg q$ case. Think the intensive longitudinal measurement setting.

**Solution**

We need to consider efficient computation of $\ell(\boldsymbol{\beta}, \boldsymbol{\Sigma}, \sigma^2)$.

Using the determinant identity shown in hw1 Q5.4 gives
$$
\begin{align*}
\vert\mathbf{\Omega}_i\vert
&= \vert\sigma^2 \mathbf{I}_{n_i} + \mathbf{Z}_i\boldsymbol{\Sigma}\mathbf{Z}_i'\vert\\
&= \det(\sigma^2 \mathbf{I}_{n_i}) \det\left[\mathbf I_q + \sigma^{-2}(\mathbf{Z}_i \mathbf{L})'\mathbf{Z}_i \mathbf{L}\right]\\
&= \sigma^{2n_i}\det(\mathbf I_q + \sigma^{-2}\mathbf{L}' \mathbf{Z}_i'\mathbf{Z}_i \mathbf{L})\\
&= \sigma^{2(n_i - q)}\det(\sigma^2\mathbf I_q + \mathbf{L}' \mathbf{Z}_i'\mathbf{Z}_i \mathbf{L}).
\end{align*}
$$

By the Cholesky decomposition, we have
$\mathbf{\Omega}_i = \sigma^2 \mathbf{I}_{n_i} + \mathbf{Z}_i \mathbf{L}\mathbf{L}' \mathbf{Z}_i' = \sigma^2 \mathbf{I}_{n_i} + \mathbf{Z}_i \mathbf{L}(\mathbf{Z}_i \mathbf{L})'$.

Using the determinant identity shown in hw1 Q5.4 gives
$$
\begin{align*}
\vert\mathbf{\Omega}_i\vert
&= \vert\sigma^2 \mathbf{I}_{n_i} + \mathbf{Z}_i \mathbf{L}(\mathbf{Z}_i \mathbf{L})'\vert\\
&= \det(\sigma^2 \mathbf{I}_{n_i}) \det\left[\mathbf I_q + \sigma^{-2}(\mathbf{Z}_i \mathbf{L})'\mathbf{Z}_i \mathbf{L}\right]\\
&= \sigma^{2n_i}\det(\mathbf I_q + \sigma^{-2}\mathbf{L}' \mathbf{Z}_i'\mathbf{Z}_i \mathbf{L})\\
&= \sigma^{2(n_i - q)}\det(\sigma^2\mathbf I_q + \mathbf{L}' \mathbf{Z}_i'\mathbf{Z}_i \mathbf{L}).
\end{align*}
$$
Hence, taking log leads to
$$
\begin{align*}
\ln\vert\mathbf{\Omega}_i\vert = (n_i - q) \ln\sigma^2 + \ln\vert\sigma^2\mathbf I_q + \mathbf{L}' \mathbf{Z}_i'\mathbf{Z}_i \mathbf{L}\vert
\end{align*}
$$

Using the Woodbury formula shown in hw1 Q5.2, we have
$$
\begin{align*}
\mathbf{\Omega}_i^{-1}
&= \sigma^{-2} \mathbf{I}_{n_i} - \sigma^{-4}\mathbf{Z}_i \mathbf{L}(\mathbf I_q + \sigma^{-2}\mathbf{L}' \mathbf{Z}_i'\mathbf{Z}_i \mathbf{L})^{-1}\mathbf{L}' \mathbf{Z}_i'\\
&= \sigma^{-2} \left[\mathbf{I}_{n_i} - \mathbf{Z}_i \mathbf{L}(\sigma^2\mathbf I_q + \mathbf{L}' \mathbf{Z}_i'\mathbf{Z}_i \mathbf{L})^{-1}\mathbf{L}' \mathbf{Z}_i'\right].
\end{align*}
$$

Additoinally, the third term without the coefficient can be expanded as follows:
$$
\begin{align*}
(\mathbf{y}_i - \mathbf{X}_i \boldsymbol{\beta})'\mathbf{\Omega}_i^{-1}(\mathbf{y}_i - \mathbf{X}_i \boldsymbol{\beta})
= \mathbf{y}_i'\mathbf{\Omega}_i^{-1}\mathbf{y}_i -2 \mathbf{y}_i'\mathbf{\Omega}_i^{-1}\mathbf{X}_i\boldsymbol{\beta} + \boldsymbol{\beta}'\mathbf{X}_i'\mathbf{\Omega}_i^{-1}\mathbf{X}_i\boldsymbol{\beta}.
\end{align*}
$$

In [33]:
function logl!(
    obs :: LmmObs{T},
    β   :: Vector{T},
    L   :: Matrix{T}, # lower triangular Cholesky factor
    σ²  :: T
    ) where T <: AbstractFloat
    n, p, q = size(obs.X, 1), size(obs.X, 2), size(obs.Z, 2)
    # compute and return the log-likelihood
    # calculate Symmetric(σ² * I + transpose(L) * obs.ztz * L)
    obs.storage_qq .= obs.ztz
    BLAS.trmm!('L', 'L', 'T', 'N', T(1), L, obs.storage_qq)
    BLAS.trmm!('R', 'L', 'N', 'N', T(1), L, obs.storage_qq)
    for j in 1:q
        obs.storage_qq[j, j] += σ²
    end

    lndetΩ = (n - q) * log(σ²) + logdet(obs.storage_qq)
    Ω⁻¹  = (I - obs.Z * L * inv(obs.storage_qq) * transpose(L) * obs.Z') / σ²
    
    storage_q .= transpose(L) * transpose(Z) * (obs.y - obs.X * β)
    logL = n * log(2π) + lndetΩ + transpose(obs.y - obs.X * β) * Ω⁻¹ * (obs.y - obs.X * β)
    # logL = - 0.5n * log(2π) - 0.5 * lndetΩ - 0.5 * (transpose(obs.y) * Ω⁻¹ * obs.y - 2 * transpose(obs.y) * Ω⁻¹ * obs.X * β + transpose(β) * transpose(obs.X) * Ω⁻¹ * obs.X * β)                
    logL *= -0.5
    return logL 
    sleep(1e-3) # wait 1 ms as if your code takes 1ms
    return zero(T)
end

logl! (generic function with 1 method)

In [34]:
function logl!(
    obs :: LmmObs{T},
    β   :: Vector{T},
    L   :: Matrix{T}, # lower triangular Cholesky factor
    σ²  :: T
    ) where T <: AbstractFloat
    n, p, q = size(obs.X, 1), size(obs.X, 2), size(obs.Z, 2)
    # compute and return the log-likelihood
    # calculate Symmetric(σ² * I + transpose(L) * obs.ztz * L)
    obs.storage_qq .= obs.ztz
    BLAS.trmm!('L', 'L', 'T', 'N', T(1), L, obs.storage_qq)
    BLAS.trmm!('R', 'L', 'N', 'N', T(1), L, obs.storage_qq)
    for j in 1:q
        obs.storage_qq[j, j] += σ²
    end

    lndetΩ = (n - q) * log(σ²) + logdet(obs.storage_qq)
    Ω⁻¹  = (I - obs.Z * L * inv(obs.storage_qq) * transpose(L) * obs.Z') / σ²
    # logL = n * log(2π) + lndetΩ + transpose(obs.y - obs.X * β) * Ω⁻¹ * (obs.y - obs.X * β)
    logL = n * log(2π) + lndetΩ + norm(obs.y - obs.X * β)^2 + norm(Ω⁻¹ * (obs.y - obs.X * β))^2
    # logL = - 0.5n * log(2π) - 0.5 * lndetΩ - 0.5 * (transpose(obs.y) * Ω⁻¹ * obs.y - 2 * transpose(obs.y) * Ω⁻¹ * obs.X * β + transpose(β) * transpose(obs.X) * Ω⁻¹ * obs.X * β)                
    logL *= -0.5
    return logL 
    sleep(1e-3) # wait 1 ms as if your code takes 1ms
    return zero(T)
end

logl! (generic function with 1 method)

**Hint**: This function shouldn't be very long. Mine, obeying 92-character rule, is 30 lines. If you find yourself writing very long code, you're on the wrong track. Think about algorithm (flop count) first then use BLAS functions to reduce memory allocations.

## Q3 Correctness (15 pts)

Compare your result (both accuracy and timing) to the [Distributions.jl](https://juliastats.org/Distributions.jl/stable/multivariate/#Distributions.AbstractMvNormal) package using following data.

In [35]:
Random.seed!(257)

# dimension
n, p, q = 2000, 5, 3
# predictors
X  = [ones(n) randn(n, p - 1)]
Z  = [ones(n) randn(n, q - 1)]
# parameter values
β  = [2.0; -1.0; rand(p - 2)]
σ² = 1.5
Σ  = fill(0.1, q, q) + 0.9I
# generate y
y  = X * β + Z * rand(MvNormal(Σ)) + sqrt(σ²) * randn(n)

# form an LmmObs object
obs = LmmObs(y, X, Z)

LmmObs{Float64}([-1.450910909560209, 1.5185224894450862, 5.265021705624027, 4.485272594164557, 0.694969966642933, 1.7723256696372405, 1.1065838446466518, 3.729166811829607, 4.288899999400642, 2.8241842645202406  …  4.058027151891634, 1.0909724390970443, 0.026692243086209766, -0.8927757653299448, 6.94725248926293, 3.5193020855673436, 4.914007299083773, 2.1610206566690797, 1.857389542694909, 6.513818951020866], [1.0 0.6790633442371218 … 0.5400611947971554 -0.632040682052606; 1.0 1.2456776800889142 … -0.4818455756130373 0.6467830314674976; … ; 1.0 0.0733124748775436 … 0.6125080259511859 0.4181258283983667; 1.0 -1.336609049786048 … -0.18567490803712938 1.0745977099307227], [1.0 -1.0193326822839996 -0.15855601254314888; 1.0 1.7462667837699666 -0.4584376230657152; … ; 1.0 1.4843185594903878 0.42458303115266854; 1.0 0.3791714762820068 0.25150666970865837], [2.347090526e-314, 2.2037039485e-314, 2.3684159804e-314, 2.2221519645e-314, 5.0e-324], [2.13922454e-314, 2.1392252513e-314, 1.265e-321], [

This is the standard way to evaluate log-density of a multivariate normal, using the Distributions.jl package. Let's evaluate the log-likelihood of this datum.

In [36]:
μ  = X * β
Ω  = Z * Σ * transpose(Z) +  σ² * I
mvn = MvNormal(μ, Symmetric(Ω)) # MVN(μ, Σ)
logpdf(mvn, y)

-3256.1793358058317

Check that your answer matches that from Distributions.jl

In [37]:
L = Matrix(cholesky(Σ).L)
logl!(obs, β, L, σ²)

-6635.55472117912

**You will lose all 15 + 30 + 30 = 75 points** if the following statement throws `AssertionError`.

In [38]:
@assert logl!(obs, β, Matrix(cholesky(Σ).L), σ²) ≈ logpdf(mvn, y)

AssertionError: AssertionError: logl!(obs, β, Matrix((cholesky(Σ)).L), σ²) ≈ logpdf(mvn, y)

## Q4 Efficiency (30 pts)

Benchmarking your code and compare to the Distributions.jl function `logpdf`.

In [39]:
# benchmark the `logpdf` function in Distribution.jl
bm1 = @benchmark logpdf($mvn, $y)

In [ ]:
# benchmark your implementation
L = Matrix(cholesky(Σ).L)
bm2 = @benchmark logl!($obs, $β, $L, $σ²)

The points you will get is
$$
\frac{x}{1000} \times 30,
$$
where $x$ is the speedup of your program against the standard method.

In [ ]:
# this is the points you'll get
clamp(median(bm1).time / median(bm2).time / 1000 * 30, 0, 30)

**Hint**: Apparently I am using 1000 as denominator because I expect your code to be at least $1000 \times$ faster than the standard method.

## Q5 Memory (30 pts)

You want to avoid memory allocation in the "hot" function `logl!`. You will lose 1 point for each `1 KiB = 1024 bytes` memory allocation. In other words, the points you get for this question is

In [ ]:
clamp(30 - median(bm2).memory / 1024, 0, 30)

**Hint**: I am able to reduce the memory allocation to 0 bytes.

## Q6 Misc (15 pts)

Coding style, Git workflow, etc. For reproducibity, make sure we (TA and myself) can run your Jupyter Notebook. That is how we grade Q4 and Q5. If we cannot run it, you will get zero points.